<a href="https://colab.research.google.com/github/alilotfi90/Flower_Classification/blob/main/test4_gaussian_histogram_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter


!pip install efficientnet_pytorch
from zipfile import ZipFile
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
import torch.optim as optim
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
import torchvision.models as models




drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:

import zipfile


with zipfile.ZipFile('/content/drive/MyDrive/flowerimages.zip', 'r') as zip_ref:
    zip_ref.extractall('unzipped_images_flower')

In [14]:
from PIL import Image
import os

unzipped_folder = '/content/unzipped_images_flower/flowers'
output_folder = '/content/resized_images_flower'


os.makedirs(output_folder, exist_ok=True)

subdirectories = [d for d in os.listdir(unzipped_folder) if os.path.isdir(os.path.join(unzipped_folder, d))]

# seen = set()
for flower_category in subdirectories:
    input_folder = os.path.join(unzipped_folder, flower_category)
    output_category_folder = os.path.join(output_folder, flower_category)


    os.makedirs(output_category_folder, exist_ok=True)

    image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]

    for image_file in image_files:
      input_image_path = os.path.join(input_folder, image_file)
      output_image_path = os.path.join(output_category_folder, image_file)

      image = Image.open(input_image_path)

      # Ensure the image is RGB
      image = image.convert("RGB")

      # Resize the image
      target_size = (224, 224)
      image = image.resize(target_size, Image.ANTIALIAS)

      image.save(output_image_path)


from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split


max_images_per_class =60
data = []

for flower_category in subdirectories:
    category_folder = os.path.join(output_folder, flower_category)
    image_files = [f for f in os.listdir(category_folder) if f.endswith('.jpg')]
    label = subdirectories.index(flower_category)  # label = folder's name

    # Limit the number of images to max_images_per_class
    image_files = image_files[:max_images_per_class]

    for image_file in image_files:
        image_path = os.path.join(category_folder, image_file)
        data.append((image_path, label))

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)





<ipython-input-14-3f11d0df2342>:33: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize(target_size, Image.ANTIALIAS)


In [11]:

# from PIL import ImageEnhance

# class HistEqualize(object):
#     def __call__(self, img):
#         if img.mode != 'L':
#             img = img.convert('L')
#         return Image.fromarray(cv2.equalizeHist(np.array(img)))

#     def __repr__(self):
#         return self.__class__.__name__ + '()'

# hist_equalize_transform = transforms.Compose([
#     HistEqualize(),
# ])

# # Display original and transformed images side-by-side
# num_images_to_display = 5
# chosen_indices = np.random.choice(len(data), num_images_to_display, replace=False)

# fig, axs = plt.subplots(num_images_to_display, 2, figsize=(10, 4*num_images_to_display))

# for i, idx in enumerate(chosen_indices):
#     img_path, _ = data[idx]
#     original_img = Image.open(img_path)
#     transformed_img = hist_equalize_transform(original_img)

#     axs[i, 0].imshow(original_img)
#     axs[i, 0].set_title("Original Image")
#     axs[i, 0].axis('off')

#     axs[i, 1].imshow(transformed_img, cmap="gray")
#     axs[i, 1].set_title("After Histogram Equalization")
#     axs[i, 1].axis('off')

# plt.tight_layout()
# plt.show()

In [6]:
# import cv2
# import numpy as np

# def estimate_noise_level(img_path):
#     image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale
#     blurred = cv2.GaussianBlur(image, (5, 5), 0)
#     noise = image - blurred
#     return np.std(noise)

# noise_levels = [estimate_noise_level(img_path) for img_path, _ in data]
# mean_noise = np.mean(noise_levels)
# std_dev_noise = np.std(noise_levels)


# min_noise = np.min(noise_levels)
# max_noise = np.max(noise_levels)
# normalized_noise_levels = [(x - min_noise) / (max_noise - min_noise) for x in noise_levels]


# std_dev_normalized_noise = np.std(normalized_noise_levels)

# print(f"Mean Estimated Noise Level: {mean_noise}")
# print(f"Standard Deviation of Noise Levels: {std_dev_noise}")
# print(f"Standard Deviation of Normalized Noise Levels: {std_dev_normalized_noise}")


Mean Estimated Noise Level: 116.96483200309474
Standard Deviation of Noise Levels: 6.352618688719487
Standard Deviation of Normalized Noise Levels: 0.11683027647538675


In [15]:
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import torchvision.transforms.functional as F
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        self.grayscale_count = 0  # Adding a counter for grayscale images

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]
        image = Image.open(image_path)

        # we don't really need the following, we changed rare grayscale images to rgb when feeding to the colour_model
        # Check and convert grayscale images
        # if image.mode != 'RGB':
        #     image = image.convert('RGB')

        if self.transform:
            image = self.transform(image)
        return image, label

class HistEqualize(object):
    def __call__(self, img):
        return F.equalize(img)

    def __repr__(self):
        return self.__class__.__name__ + '()'

#base transform
transform_gray = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])
])
# Gaussian Blur Transformation
transform_gaussian = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.GaussianBlur(5),  # Kernel size of 5, you can adjust as needed
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])
])

# Histogram Equalization Transformation
transform_hist_eq = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    HistEqualize(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])
])

# Data Loaders for the new transformations
train_loader_gray = DataLoader(CustomDataset(train_data, transform=transform_gray), batch_size=32, shuffle=True)
test_loader_gray = DataLoader(CustomDataset(test_data, transform=transform_gray), batch_size=32)
train_loader_gaussian = DataLoader(CustomDataset(train_data, transform=transform_gaussian), batch_size=32, shuffle=True)
test_loader_gaussian = DataLoader(CustomDataset(test_data, transform=transform_gaussian), batch_size=32)
train_loader_hist_eq = DataLoader(CustomDataset(train_data, transform=transform_hist_eq), batch_size=32, shuffle=True)
test_loader_hist_eq = DataLoader(CustomDataset(test_data, transform=transform_hist_eq), batch_size=32)

class CustomEfficientNet(nn.Module):
    def __init__(self, pretrained_model_name='efficientnet-b0', num_classes=16):
        super(CustomEfficientNet, self).__init__()
        self.embedding = nn.Conv2d(1, 3, kernel_size=1, stride=1, padding=0)
        self.eff_net = EfficientNet.from_pretrained(pretrained_model_name, num_classes=num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.eff_net(x)
        return x
model_gray = CustomEfficientNet().to(device)  # For regular grayscale
model_gray_gaussian = CustomEfficientNet().to(device)  # For Gaussian-blurred grayscale
model_gray_hist_eq = CustomEfficientNet().to(device)  # For histogram-equalized grayscale


optimizer_gray = optim.SGD(model_gray.parameters(), lr=0.001, momentum=0.9)
optimizer_gray_gaussian = optim.SGD(model_gray_gaussian.parameters(), lr=0.001, momentum=0.9)
optimizer_gray_hist_eq = optim.SGD(model_gray_hist_eq.parameters(), lr=0.001, momentum=0.9)

criterion = nn.CrossEntropyLoss()

def train_test_model(model, train_loader, test_loader, optimizer, criterion, epochs_list, model_name):
    accuracies = []
    cumulative_epochs = 0
    for epochs in epochs_list:
        cumulative_epochs += epochs
        for epoch in range(epochs):
            model.train()
            running_loss = 0.0
            for i, data in enumerate(train_loader, 0):
                inputs, labels = data[0].to(device), data[1].to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                if i % 10 == 9:
                    print(f'[{cumulative_epochs + epoch + 1}, {i + 1}] loss: {running_loss / 10:.3f}')
                    running_loss = 0.0

            model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for data in test_loader:
                    images, labels = data[0].to(device), data[1].to(device)
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            accuracy = 100 * correct / total
            accuracies.append(accuracy)
            print(f'Accuracy of {model_name} after {cumulative_epochs + epoch + 1} epochs on test images: {accuracy:.2f}%')
    return accuracies




Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0


In [17]:
epochs_list = [10, 20,30,40]
# Training and geting accuracies

train_loader_gray = DataLoader(CustomDataset(train_data, transform=transform_gray), batch_size=32, shuffle=True)
test_loader_gray = DataLoader(CustomDataset(test_data, transform=transform_gray), batch_size=32)
train_loader_gaussian = DataLoader(CustomDataset(train_data, transform=transform_gaussian), batch_size=32, shuffle=True)
test_loader_gaussian = DataLoader(CustomDataset(test_data, transform=transform_gaussian), batch_size=32)
train_loader_hist_eq = DataLoader(CustomDataset(train_data, transform=transform_hist_eq), batch_size=32, shuffle=True)
test_loader_hist_eq = DataLoader(CustomDataset(test_data, transform=transform_hist_eq), batch_size=32)

# accuracies_hist_eq = train_test_model(model_gray_hist_eq, train_loader_hist_eq, test_loader_hist_eq, optimizer_gray, criterion, epochs_list, "Histogram Equalization Model")
# accuracies_gaussian = train_test_model(model_gray_gaussian, train_loader_gaussian, test_loader_gaussian, optimizer_gray, criterion, epochs_list, "Gaussian Model")



accuracies_gaussian = train_test_model(model_gray_gaussian, train_loader_gaussian, test_loader_gaussian, optimizer_gray_gaussian, criterion, epochs_list, "Gaussian Model")
accuracies_hist_eq = train_test_model(model_gray_hist_eq, train_loader_hist_eq, test_loader_hist_eq, optimizer_gray_hist_eq, criterion, epochs_list, "Histogram Equalization Model")
accuracies_gray = train_test_model(model_gray, train_loader_gray, test_loader_gray, optimizer_gray, criterion, epochs_list, "Grayscale Model")




df = pd.DataFrame({
    'Epochs': [sum(epochs_list[:i+1]) for i in range(len(epochs_list))],
    'Grayscale_Model_Accuracy': accuracies_gray,
    'Gaussian_Model_Accuracy': accuracies_gaussian,
    'Histogram_Model_Accuracy': accuracies_hist_eq,
})

import matplotlib.pyplot as plt

fig, ax =plt.subplots(figsize=(12,4))
ax.axis('tight')
ax.axis('off')
the_table = ax.table(cellText=df.values,colLabels=df.columns,loc='center')
the_table.auto_set_font_size(False)
the_table.set_fontsize(10)
plt.savefig("/content/drive/MyDrive/model_accuracies.jpeg")

[11, 10] loss: 2.779
[11, 20] loss: 2.740
Accuracy of Gaussian Model after 11 epochs on test images: 8.33%
[12, 10] loss: 2.681
[12, 20] loss: 2.642
Accuracy of Gaussian Model after 12 epochs on test images: 11.98%
[13, 10] loss: 2.561
[13, 20] loss: 2.537
Accuracy of Gaussian Model after 13 epochs on test images: 16.15%
[14, 10] loss: 2.468
[14, 20] loss: 2.406
Accuracy of Gaussian Model after 14 epochs on test images: 22.92%
[15, 10] loss: 2.335
[15, 20] loss: 2.302
Accuracy of Gaussian Model after 15 epochs on test images: 31.77%
[16, 10] loss: 2.205
[16, 20] loss: 2.207
Accuracy of Gaussian Model after 16 epochs on test images: 35.42%
[17, 10] loss: 2.138
[17, 20] loss: 2.045
Accuracy of Gaussian Model after 17 epochs on test images: 41.67%
[18, 10] loss: 1.958
[18, 20] loss: 1.974
Accuracy of Gaussian Model after 18 epochs on test images: 44.27%
[19, 10] loss: 1.875
[19, 20] loss: 1.810
Accuracy of Gaussian Model after 19 epochs on test images: 50.52%
[20, 10] loss: 1.794
[20, 20]

ValueError: ignored

In [20]:
print(max(accuracies_gray))
print(max(accuracies_gaussian))
print(max(accuracies_hist_eq))


80.20833333333333
78.125
79.6875
